In [64]:
#Python Default 

import pickle

#Essentials

import time
import numpy as np

#from objproxies import *

import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

#Qiskit

from qiskit.providers.models import *
from qiskit_ibm_runtime.fake_provider import *
from qiskit.providers.fake_provider import *

## Qiskit Nature
    
from qiskit_nature.units import *   #ANGSTROM

from qiskit_nature.second_q.drivers import *    #PySCFDriver

from qiskit_nature.second_q.mappers import * #JordanWignerMapper

from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit_nature.second_q.problems import ElectronicStructureProblem

from qiskit_nature.second_q.circuit.library import *  #Ansatz, HF

## Qiskit Algorithms
    
from qiskit_algorithms.minimum_eigensolvers import *    #VQE
 
from qiskit_algorithms.optimizers import *    #SLSQP

from qiskit.circuit.library import EfficientSU2   #EfficientSU2

## Qiskit Primitives

from qiskit.primitives import Estimator as Estimator_Nature

from qiskit_aer.primitives.estimator import Estimator as Estimator_Aer

## Qiskit Noise Models

from qiskit_aer.noise import NoiseModel

from qiskit.providers.fake_provider import *

##Qiskit Runtime IBM

from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options

from qiskit_ibm_runtime import Estimator as Estimator_Qiskit    #Estimator

%matplotlib widget


In [71]:
service = QiskitRuntimeService()
backend = service.backend("ibm_brisbane")
noise_model = NoiseModel.from_backend(backend)

In [66]:
def salvar(a):
    with open(f'Códigos/Gabriel/GroundState6/dados/{a}.pickle', 'wb') as f:
        pickle.dump(eval(a), f) 

def abrir(a):
    with open(f'Códigos/Gabriel/GroundState6/dados/{a}.pickle', 'rb') as f:
        return pickle.load(f)

In [67]:
def Aer_Estimator(intervalo, base = "sto3g", optimizer = SLSQP(), session = None, noise = None, mapper = JordanWignerMapper(), shots = 2048, seed = None, method = "automatic"):
    
    for a in intervalo: 
        
        driver = PySCFDriver(
            atom= f"H 0 0 0; H 0 0 {a}",
            basis=base,
            charge=0,
            spin=0,
            unit=DistanceUnit.ANGSTROM,
        )
        
        es_problem = driver.run()

        options = ''

        options_ = [f'run_options = {{"shots": {shots}, "seed": {seed}}}']
                    
        if shots == None:
            
            options_.append('approximation = True')
        
        if session != None:
            
            options_.append(f'session = {session}')
        
        if seed != None:
            
            options_.append(f'transpile_options = {{"seed_transpiler": {seed}}}')

        if noise != None:
            
            options_.append(f'backend_options = {{"coupling_map": {noise}().coupling_map, "noise_model": NoiseModel.from_backend({noise}())}}')
        
        for b in options_:
            
            options += b + ','
        
        options = options[:-1]

        estimator = eval(f'Estimator_Aer({options})')
        
        mapper = mapper
        
        ansatz = UCCSD(
            es_problem.num_spatial_orbitals,
            es_problem.num_particles,
            mapper,
            initial_state=HartreeFock(
                es_problem.num_spatial_orbitals,
                es_problem.num_particles,
                mapper,
            ),
        )

        vqe_solver = VQE(estimator, ansatz, optimizer)
        vqe_solver.initial_point = [0.0] * ansatz.num_parameters
        
        calc = GroundStateEigensolver(mapper, vqe_solver)
        
        res = calc.solve(es_problem)
        
        yield [a, res.groundenergy + res.nuclear_repulsion_energy]

In [74]:
def Estimator_intervalo(intervalo, base = "sto3g", optimizer = COBYLA(), noise = None, backend = None, session = None, mapper = JordanWignerMapper(), shots = 2048, seed = 1):
        
    for a in intervalo: 
        
        driver = PySCFDriver(
            atom= f"H 0 0 0; H 0 0 {a}",
            basis=base,
            charge=0,
            spin=0,
            unit=DistanceUnit.ANGSTROM,
        )
        
        es_problem = driver.run()
        
        hamiltonian = es_problem.hamiltonian
    

        if session != None and backend != None:
            
            estimator = Estimator_Qiskit(backend = backend, session = session, options = {'shots': shots})
        
        elif noise != None:
            
            estimator = Estimator_Aer(approximation=True, backend_options = {'noise_model':noise}, run_options = {'shots': shots, 'seed' : seed})
            
        else:
            
            estimator = Estimator_Nature(options = {'shots': shots})
        
        mapper = mapper
        
        ansatz = UCCSD(
            es_problem.num_spatial_orbitals,
            es_problem.num_particles,
            mapper,
            initial_state=HartreeFock(
                es_problem.num_spatial_orbitals,
                es_problem.num_particles,
                mapper,
            ),
        )

        vqe_solver = VQE(estimator, ansatz, optimizer)
        vqe_solver.initial_point = [0.0] * ansatz.num_parameters
        
        calc = GroundStateEigensolver(mapper, vqe_solver)
        
        res = calc.solve(es_problem)
        
        yield [a, res.groundenergy + res.nuclear_repulsion_energy]

In [75]:
list(Estimator_intervalo([1], noise = noise_model, shots = 100, seed = 1))

[[1, -0.8593641372611985]]

In [ ]:
intervalo_total = np.concatenate(((intervalo := np.concatenate(((intervalo_importante := np.linspace(0.1, (end := 1.5), 15)), np.linspace(end, 3, 10)))), (intervalo_distante := np.linspace(20, 30, 10))))

for a in (metodos := range(61, 100, 10)):
    
   exec(f"estimator_qiskit_shots_{a} = np.array(list(Estimator_intervalo(intervalo, shots = {a}, session=session, seed = 1)))")
   exec(f'estimator_qiskit_shots_{a}[:, 1] -= (error := - np.polyfit(intervalo_distante, np.array(list(Estimator_intervalo(intervalo_distante, shots = {a}, session=session, seed = 1)))[:,1], 0))')
   
   exec(f"estimator_qiskit_essencial_{a} = np.array(list(Estimator_intervalo(np.linspace((minimo := min(estimator_qiskit_shots_{a}, key = lambda x: x[1])[0]) + (erro := minimo * 10/100), minimo - erro, 10), shots = {a}, session=session, seed = 1)))")
   exec(f"estimator_qiskit_essencial_{a}[:, 1] -= error")
   
   exec(f"estimator_qiskit_shots_total_{a} = np.concatenate((estimator_qiskit_shots_{a}, estimator_qiskit_essencial_{a}))")
   exec(f"estimator_qiskit_shots_total_{a} = estimator_qiskit_shots_total_{a}[np.argsort(estimator_qiskit_shots_total_{a}[:, 0])]")
   
   salvar(f'estimator_qiskit_shots_{a}')
   salvar(f'estimator_qiskit_shots_total_{a}')
   salvar(f'estimator_qiskit_essencial_{a}')

In [76]:
list(Estimator_intervalo([1], noise = FakeVigoV2, shots = 100, seed = 1))

AttributeError: type object 'FakeVigoV2' has no attribute 'basis_gates'

In [20]:
intervalo_total = np.concatenate(((intervalo := np.concatenate(((intervalo_importante := np.linspace(0.1, (end := 1.5), 15)), np.linspace(end, 3, 10)))), (intervalo_distante := np.linspace(20, 30, 10))))
   

for a in (metodos := range(60, 100, 10)):
    
   exec(f"estimator_qiskit_{a} = np.array(list(Estimator_intervalo(intervalo, shots = {a}, seed = 1)))")
   exec(f'estimator_qiskit_{a}[:, 1] -= (error := - np.polyfit(intervalo_distante, np.array(list(Estimator_intervalo(intervalo_distante, shots = {a}, seed = 1)))[:,1], 0))')
   
   exec(f"estimator_qiskit_essencial_{a} = np.array(list(Estimator_intervalo(np.linspace((minimo := min(estimator_qiskit_shots_{a}, key = lambda x: x[1])[0]) + (erro := minimo * 10/100), minimo - erro, 10), shots = {a}, seed = 1)))")
   exec(f"estimator_qiskit_essencial_{a}[:, 1] -= error")
   
   exec(f"estimator_qiskit_total_{a} = np.concatenate((estimator_qiskit_shots_{a}, estimator_qiskit_essencial_{a}))")
   exec(f"estimator_qiskit_total_{a} = estimator_qiskit_shots_total_{a}[np.argsort(estimator_qiskit_shots_total_{a}[:, 0])]")
   
   salvar(f'estimator_qiskit_{a}')
   salvar(f'estimator_qiskit_shots_total_{a}')
   salvar(f'estimator_qiskit_essencial_{a}')

AccountNotFoundError: 'Unable to find account.'

In [77]:
from qiskit.providers.fake_provider import FakeVigo

ImportError: cannot import name 'FakeVigo' from 'qiskit.providers.fake_provider' (/opt/anaconda3/envs/VASP/lib/python3.10/site-packages/qiskit/providers/fake_provider/__init__.py)